# Bibliotecas

In [ ]:
from obterDados import obterSimboloPosicao, obterSimboloData
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow as tf
import gymnasium as gym
import gym_anytrading
from stable_baselines3 import A2C, PPO
from json import dumps

# Obtendo dados de M5 e D1

In [ ]:
rawDataM5 = obterSimboloPosicao()[:-100*108]
rawDataM5

In [ ]:
rawDataM5.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
rawDataM5

In [ ]:
rawDataD1 = obterSimboloData(rawDataM5.index[-1], n=int(len(rawDataM5)/108)+200)
rawDataD1

In [ ]:
rawDataD1.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
rawDataD1

# Criando ambiente e testando aleatoriamente

In [ ]:
from customTradingEnv2 import CustomTradingEnv

env = CustomTradingEnv(df_5min=rawDataM5, df_daily=rawDataD1)

In [ ]:
# observation = env.reset()
# done = False
# cont = 0
# while not done:
#     action = env.action_space.sample()
#     observation, reward, done, info = env.step(action)
#     print(observation)

# trades = info['trades']

# Treinando modelo com aprendizado por reforço

In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback

modelo = PPO('MultiInputPolicy', env, verbose=1, tensorboard_log='logs/ppo/')

models_dir = 'models/PPO'
checkpointCallback = CheckpointCallback(save_freq=10000, save_path=models_dir)
modelo.learn(total_timesteps=len(rawDataM5)*200, callback=checkpointCallback)

### Continuar treinamento a partir de modelo salvo

In [ ]:
# from stable_baselines3.common.callbacks import CheckpointCallback

# models_dir = 'models/PPO'
# model_path = f"{models_dir}/rl_model_4850000_steps.zip"
# modelo = PPO.load(model_path, env=env)
# checkpointCallback = CheckpointCallback(save_freq=10000, save_path=models_dir)
# modelo.learn(total_timesteps=len(rawDataM5)*500-4850000, callback=checkpointCallback, reset_num_timesteps=False)

# Testando modelo treinado

In [ ]:
testDataM5 = obterSimboloPosicao()[-(20*108+300):]
testDataM5.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
testDataD1 = obterSimboloData(testDataM5.index[-1], n=int(len(testDataM5)/108)+200)
testDataD1.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)

In [ ]:
from customTradingEnv2 import CustomTradingEnv

env = CustomTradingEnv(df_5min=testDataM5, df_daily=testDataD1)

In [ ]:
models_dir = 'models/PPO'
model_path = f"{models_dir}/rl_model_14010000_steps.zip"
modelo = PPO.load(model_path, env=env)

In [ ]:
observation = env.reset()
done = False
while not done:
    action = modelo.predict(observation)
    observation, reward, done, info = env.step(int(action[0]))
    # env.render()

info